In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_lfw_people

# Load the LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw_people.images
y = lfw_people.target

# Preprocess the data
X = X.astype('float32') / 255.0
X = np.expand_dims(X, axis=-1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 37, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_encoded, epochs=10, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print("Test Accuracy:", test_accuracy)

# Save the trained model
model.save("lfw_facial_recognition_model.h5")


Epoch 1/10
26/26 [==============================] - 3s 83ms/step - loss: 2.0172 - accuracy: 0.3811 - val_loss: 1.7147 - val_accuracy: 0.4029
Epoch 2/10
26/26 [==============================] - 3s 124ms/step - loss: 1.7473 - accuracy: 0.3981 - val_loss: 1.6818 - val_accuracy: 0.4029
Epoch 3/10
26/26 [==============================] - 2s 74ms/step - loss: 1.7299 - accuracy: 0.3981 - val_loss: 1.6985 - val_accuracy: 0.4029
Epoch 4/10
26/26 [==============================] - 2s 74ms/step - loss: 1.7326 - accuracy: 0.3981 - val_loss: 1.6838 - val_accuracy: 0.4029
Epoch 5/10
26/26 [==============================] - 2s 72ms/step - loss: 1.7316 - accuracy: 0.3981 - val_loss: 1.6761 - val_accuracy: 0.4029
Epoch 6/10
26/26 [==============================] - 2s 73ms/step - loss: 1.7250 - accuracy: 0.3981 - val_loss: 1.6723 - val_accuracy: 0.4029
Epoch 7/10
26/26 [==============================] - 2s 71ms/step - loss: 1.7258 - accuracy: 0.3981 - val_loss: 1.6832 - val_accuracy: 0.4029
Epoch 8/10
2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**implementing a classical deep learning model (e.g., using TensorFlow or PyTorch) to train on the facial recognition dataset and achieve high accuracy in identifying faces. Optimize the model architecture, hyperparameters, and training process to improve the recognition performance.**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize

# Load LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# Preprocess the data
X = lfw_people.images
y = lfw_people.target

# Resize images to match ResNet50 input shape
X_resized = np.array([resize(img, (128, 128, 3)) for img in X])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resized, y, test_size=0.2, random_state=42)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Load pre-trained ResNet50 model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze pre-trained layers
for layer in model.layers:
    layer.trainable = False

# Add custom output layers
x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
predictions = tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')(x)

# Create the final model
final_model = tf.keras.models.Model(inputs=model.input, outputs=predictions)

# Compile the model
final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
final_model.fit(X_train, y_train_encoded, epochs=10, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = final_model.evaluate(X_test, y_test_encoded)
print("Test Accuracy:", test_accuracy)



Epoch 1/10
26/26 [==============================] - 76s 3s/step - loss: 1.8266 - accuracy: 0.3350 - val_loss: 1.6807 - val_accuracy: 0.4029
Epoch 2/10
26/26 [==============================] - 69s 3s/step - loss: 1.7195 - accuracy: 0.3981 - val_loss: 1.6838 - val_accuracy: 0.4029
Epoch 3/10
26/26 [==============================] - 71s 3s/step - loss: 1.7348 - accuracy: 0.3981 - val_loss: 1.6818 - val_accuracy: 0.4029
Epoch 4/10
26/26 [==============================] - 69s 3s/step - loss: 1.7239 - accuracy: 0.3981 - val_loss: 1.6717 - val_accuracy: 0.4029
Epoch 5/10
26/26 [==============================] - 60s 2s/step - loss: 1.7120 - accuracy: 0.3981 - val_loss: 1.6829 - val_accuracy: 0.4029
Epoch 6/10
26/26 [==============================] - 71s 3s/step - loss: 1.7158 - accuracy: 0.3981 - val_loss: 1.6660 - val_accuracy: 0.4029
Epoch 7/10
26/26 [==============================] - 69s 3s/step - loss: 1.7142 - accuracy: 0.3981 - val_loss: 1.6641 - val_accuracy: 0.4029
Epoch 8/10
26/26 [==

**Developing a Python program that utilizes a deep learning model (such as a convolutional neural network) to perform facial recognition on a dataset of images. The program should be able to accurately identify and classify faces in the images based on pre-trained facial recognition features.**